In [51]:
from TPNN.tools.CNNarchitecture import *
from TPNN.tools.activation_functions import *
import numpy as np

## Test net creation

In [52]:
# create net:
cnnLayer = CNNlayer((1, 10, 10), sigmoid, 2, (3,3), 2)
poolingLayer = MaxPoolingLayer((2,2), cnnLayer.output_shape)
reformatLayer = ReformatLayer(poolingLayer.output_shape)
denseLayer = DenseLayer(sigmoid, 3, reformatLayer.output_shape[0])

net = Net()
net.add_layer(cnnLayer)
net.add_layer(poolingLayer)
net.add_layer(reformatLayer)
net.add_layer(denseLayer)
net.print_config()

Layers count: 4
layers configuration: 

CNN layer:
 input shape - (1, 10, 10); output shape - (2, 4, 4)
 cores: 
[0]:
     -0.5645104054924313 0.947757833676552 0.3400289010757025 
     -0.6039922171199346 -0.18006999664636925 0.6617296491069578 
     0.350531281086518 0.588301635730271 0.048079844973535124 
[1]:
     -0.545187821088472 0.3057110774409122 0.46071706974092863 
     0.3078655908897401 -0.1465924846455 -0.8242311690199027 
     -0.9494929106744749 0.9355691115765885 -0.34175316986069193 
 biases: 
     0.21357940447888657 0.9916882173805484 

MaxPooling layer
 input shape - (2, 4, 4); output shape - (2, 2, 2)
 core shape: (2, 2)

Reformat layer
 input shape - (2, 2, 2); output shape - (4,)

Dense layer
 neuron count: 3
 input shape - (4,); output shape - (3,)
 weight matrix:
     -0.963086552191071 0.8293279061917926 0.3929903270803585 
     0.10591233221894059 0.15122334978944907 -0.8139697841575544 
     -0.7074388195227352 -0.14666053753006936 0.75949199855387 
     -0

In [53]:
input_data = np.random.rand(1, 10, 10)

print("input:")
print(input_data)

input:
[[[0.97735052 0.68632112 0.54846308 0.60955176 0.97581207 0.44010814
   0.39392323 0.67478709 0.67068699 0.86211966]
  [0.29571297 0.61133697 0.45819339 0.19775778 0.70370774 0.57823846
   0.11236272 0.56672691 0.71216147 0.42781545]
  [0.59620981 0.310804   0.00532547 0.20080091 0.88028108 0.33174751
   0.79135657 0.26744905 0.80340613 0.12132757]
  [0.66654442 0.15251911 0.79732332 0.57224524 0.78793499 0.27632656
   0.60792533 0.15168695 0.77573092 0.09684585]
  [0.66344466 0.4318951  0.87119421 0.19008512 0.81630098 0.50360995
   0.56109283 0.32893213 0.62993905 0.66558952]
  [0.18916676 0.61530595 0.86930323 0.68499748 0.57380879 0.0446615
   0.71084793 0.10560853 0.73673834 0.59053198]
  [0.54837592 0.92933554 0.70699658 0.37182783 0.62114331 0.83043053
   0.09785501 0.98353842 0.53412408 0.99534064]
  [0.40332625 0.70037001 0.32865235 0.18552322 0.51831157 0.46689949
   0.1181837  0.54865155 0.4133837  0.35877793]
  [0.40942772 0.64363565 0.55370947 0.34531896 0.14075146 

In [54]:
print(net.get_output(input_data))

[0.27230801 0.81363662 0.60584215]


## Test CNN layer output

In [55]:
def print_info(input, net):
    print("input:")
    print(input)
    print("net config")
    net.print_config()

cnnLayer = CNNlayer((2,4,4), ident, 2, (2,2), 2)
cores = np.array([np.eye(2), np.eye(2)])
cnnLayer.set_cores(cores)
cnnLayer.set_biases(np.array([0, 0]))

net = Net()
net.add_layer(cnnLayer)

input = np.array([np.eye(4),np.eye(4)])

print_info(input, net)

input:
[[[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]

 [[1. 0. 0. 0.]
  [0. 1. 0. 0.]
  [0. 0. 1. 0.]
  [0. 0. 0. 1.]]]
net config
Layers count: 1
layers configuration: 

CNN layer:
 input shape - (2, 4, 4); output shape - (2, 2, 2)
 cores: 
[0]:
     1.0 0.0 
     0.0 1.0 
[1]:
     1.0 0.0 
     0.0 1.0 
 biases: 
     0 0 



In [56]:
print(net.get_output(input))

[[[4. 0.]
  [0. 4.]]

 [[4. 0.]
  [0. 4.]]]


## Test pooling layer

In [57]:
net = Net()
poolingLayer = MaxPoolingLayer((2,2), (2,4,4))
net.add_layer(poolingLayer)

input = np.array([[[1,2,1,2], [1,2,1,2], [1,2,1,2], [1,2,1,2]], [[1,2,1,2], [1,2,1,2], [1,2,1,2], [1,2,1,2]]])

print_info(input, net)

input:
[[[1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]]

 [[1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]
  [1 2 1 2]]]
net config
Layers count: 1
layers configuration: 

MaxPooling layer
 input shape - (2, 4, 4); output shape - (2, 2, 2)
 core shape: (2, 2)



In [58]:
print(net.get_output(input))

[[[2. 2.]
  [2. 2.]]

 [[2. 2.]
  [2. 2.]]]


## Test dense layer

In [59]:
denseLayer = DenseLayer(ident, 10, 10)
denseLayer.set_weighs(np.eye(10))
denseLayer.set_biases(np.zeros((10, 1)))

net = Net()
net.add_layer(denseLayer)

input = np.arange(0,10)

print_info(input, net)

input:
[0 1 2 3 4 5 6 7 8 9]
net config
Layers count: 1
layers configuration: 

Dense layer
 neuron count: 10
 input shape - (10,); output shape - (10,)
 weight matrix:
     1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 0.0 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 1.0 
 biases vector: 
     0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 0.0 



In [60]:
net.get_output(input)

array([0., 1., 2., 3., 4., 5., 6., 7., 8., 9.])

## Test reformat layer

In [61]:
net = Net()
reformatLayer = ReformatLayer((2, 4, 4))
net.add_layer(reformatLayer)

input = np.ones((2,4,4))

print_info(input, net)

input:
[[[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]

 [[1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]
  [1. 1. 1. 1.]]]
net config
Layers count: 1
layers configuration: 

Reformat layer
 input shape - (2, 4, 4); output shape - (16,)



In [62]:
net.get_output(input)

array([2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.])